In [1]:
! pip install gdown 
! gdown 1z28MmoZmBd46FRF4hmFkmLIFemqxg5C8
! unzip -o jigsaw-toxic-comment-classification-challenge.zip
! rm -f jigsaw-toxic-comment-classification-challenge.zip
! unzip -o train.csv.zip -d /content/ 
! unzip -o test.csv.zip -d /content/ 
#https://drive.google.com/file/d/1z28MmoZmBd46FRF4hmFkmLIFemqxg5C8/view?usp=sharing

zsh:1: command not found: pip
zsh:1: command not found: gdown
unzip:  cannot find or open jigsaw-toxic-comment-classification-challenge.zip, jigsaw-toxic-comment-classification-challenge.zip.zip or jigsaw-toxic-comment-classification-challenge.zip.ZIP.
Archive:  train.csv.zip
checkdir:  cannot create extraction directory: /content
           Read-only file system
Archive:  test.csv.zip
checkdir:  cannot create extraction directory: /content
           Read-only file system


In [2]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [133]:
import pandas as pd
import re
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pprint import pprint
import numpy as np
from  gensim.utils import simple_preprocess
import joblib
from IPython.display import clear_output

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
from nltk import word_tokenize

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import multilabel_confusion_matrix

import gensim.downloader as trained_models
import os.path as path

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer

from sklearn.model_selection import RandomizedSearchCV

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, GridSearchCV

import warnings
warnings.filterwarnings('ignore') # supress warnings

In [134]:
#### extract all the zip files to unprocessed folder

! unzip -o ./train.csv.zip -d ./unprocessed
! unzip -o ./test.csv.zip -d ./unprocessed
! unzip -o ./test_labels.csv.zip -d ./unprocessed

clear_output(False)

In [172]:
englishStopWords = list(stopwords.words('english'))
englishStopWords.extend(['utc','talk'])
englishStopWords = set(englishStopWords)
lemmatizer = WordNetLemmatizer()

nDAY = r'(?:[0-3]?\d)'  # day can be from 1 to 31 with a leading zero 
nMNTH = r'(?:11|12|10|0?[1-9])' # month can be 1 to 12 with a leading zero
nYR = r'(?:(?:19|20)\d\d)'  # I've restricted the year to being in 20th or 21st century on the basis 
                            # that people doon't generally use all number format for old dates, but write them out 
nDELIM = r'(?:[\/\-\._])?'  # 
NUM_DATE = f"""
    (?P<num_date>
        (?:^|\D) # new bit here
        (?:
        # YYYY-MM-DD
        (?:{nYR}(?P<delim1>[\/\-\._]?){nMNTH}(?P=delim1){nDAY})
        |
        # YYYY-DD-MM
        (?:{nYR}(?P<delim2>[\/\-\._]?){nDAY}(?P=delim2){nMNTH})
        |
        # DD-MM-YYYY
        (?:{nDAY}(?P<delim3>[\/\-\._]?){nMNTH}(?P=delim3){nYR})
        |
        # MM-DD-YYYY
        (?:{nMNTH}(?P<delim4>[\/\-\._]?){nDAY}(?P=delim4){nYR})
        )
        (?:\D|$) # new bit here
    )"""
DAY = r"""
(?:
    # search 1st 2nd 3rd etc, or first second third
    (?:[23]?1st|2{1,2}nd|\d{1,2}th|2?3rd|first|second|third|fourth|fifth|sixth|seventh|eighth|nineth)
    |
    # or just a number, but without a leading zero
    (?:[123]?\d)
)"""
MONTH = r'(?:january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec)'
YEAR = r"""(?:(?:[12]?\d|')?\d\d)"""
DELIM = r'(?:\s*(?:[\s\.\-\\/,]|(?:of))\s*)'

YEAR_4D = r"""(?:[12]\d\d\d)"""
DATE_PATTERN = f"""(?P<wordy_date>
    # non word character or start of string
    (?:^|\W)
        (?:
            # match various combinations of year month and day 
            (?:
                # 4 digit year
                (?:{YEAR_4D}{DELIM})?
                    (?:
                    # Day - Month
                    (?:{DAY}{DELIM}{MONTH})
                    |
                    # Month - Day
                    (?:{MONTH}{DELIM}{DAY})
                    )
                # 2 or 4 digit year
                (?:{DELIM}{YEAR})?
            )
            |
            # Month - Year (2 or 3 digit)
            (?:{MONTH}{DELIM}{YEAR})
        )
    # non-word character or end of string
    (?:$|\W)
)"""

TIME = r"""(?:
(?:
# first number should be 0 - 59 with optional leading zero.
[012345]?\d
# second number is the same following a colon
:[012345]\d
)
# next we add our optional seconds number in the same format
(?::[012345]\d)?
# and finally add optional am or pm possibly with . and spaces
(?:\s*(?:a|p)\.?m\.?)?
)"""

COMBINED = f"""(?P<combined>
    (?:
        # time followed by date, or date followed by time
        {TIME}?{DATE_PATTERN}{TIME}?
        |
        # or as above but with the numeric version of the date
        {TIME}?{NUM_DATE}{TIME}?
    ) 
    # or a time on its own
    |
    (?:{TIME})
)"""

regexDate = re.compile(COMBINED, re.IGNORECASE | re.VERBOSE | re.UNICODE)


def removeNumber(comment):
    return re.sub(r'\d+',"",comment)

def removeDoubleDots(comment):
    return re.sub(r'\.{2,}',"",comment)

def removeDoubleSpace(comment):
    return re.sub(r'[ ]{2,}'," ",comment)

def removePunctuations(comment):
    return re.sub(r'[^\w\s]','',comment)

def removeNewLine(comment):
    return re.sub(r'\n'," ",comment)

def removeURLs(comment):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', comment, flags=re.MULTILINE)

def removeDate(comment):
    return regexDate.sub(' ',comment)

def removeRepeatingWords(comment):
    return re.sub(r'(\w+)( \1)+', r'\1', comment, flags=re.IGNORECASE)

def lemmatize(comment):
    tokens = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(comment) if word not in englishStopWords]
    return " ".join(tokens)


def processComment(comment):
    comment = removeURLs(comment)
    comment = removeDate(comment)
    comment = removeNumber(comment)
    comment = removeNewLine(comment)
    comment = removeDoubleSpace(comment)
    comment = removeDoubleDots(comment)
    comment = removePunctuations(comment)
    comment = lemmatize(comment)
    comment = removeRepeatingWords(comment)
    return comment.lower()    




In [135]:
class TextCleaner(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.apply(lambda x : processComment(x))

In [195]:
class ParagraphVectorizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        dataset = './processed/w2vEmbeddings.model'
        self.wv = KeyedVectors.load(dataset, mmap='r')
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        finalVector = []
        for paragraph in X:
            pVector = []
            tokens = word_tokenize(paragraph)
            for token in tokens:
                if(self.wv.__contains__(token)):
                    pVector.append(self.wv[token])
            if len(pVector) > 0:
                pVector = np.average(pVector,axis=0).tolist()
            else:
                pVector = np.zeros(100)
            finalVector.append(pVector)
        return pd.DataFrame(finalVector)

In [174]:
# create pre processed data directory to optimize the preprocessing


def mergeTrainCommentAndLabels(dfComment, dfLabels):
    for index, row in dfComment.iterrows():
        index = row.name
        targetRow = dfLabels.iloc[index]
        if row.id != targetRow.id:
            print("Not matching")

        dfComment.at[index, 'toxic'] = int(targetRow.toxic)
        dfComment.at[index, 'severe_toxic'] = int(targetRow.severe_toxic)
        dfComment.at[index, 'obscene'] = int(targetRow.obscene)
        dfComment.at[index, 'threat'] = int(targetRow.threat)
        dfComment.at[index, 'insult'] = int(targetRow.insult)
        dfComment.at[index, 'identity_hate'] = int(targetRow.identity_hate)
    finalDf = dfComment.copy(deep=True)
    finalDf = finalDf.drop(finalDf[finalDf.toxic == -1].index)
    finalDf['comment_text'] = finalDf['comment_text'].apply( lambda x: processComment(x))
    return finalDf

if not os.path.exists('./processed'):
    os.makedirs('./processed')

if not os.path.exists('./processed/train.csv'):
    dataset = pd.read_csv('./unprocessed/train.csv')
    dataset['comment_text'] = dataset['comment_text'].apply(
        lambda x: processComment(x))
    dataset = dataset.dropna()
    dataset.to_csv('./processed/train.csv')

if not os.path.exists('./processed/test.csv'):
    x = pd.read_csv('./unprocessed/test.csv')
    y = pd.read_csv('./unprocessed/test_labels.csv')
    finalDf = mergeTrainCommentAndLabels(x,y)
    finalDf = finalDf.dropna()
    finalDf.to_csv('./processed/test.csv')


In [198]:
trainData = pd.read_csv('./processed/train.csv')
testData = pd.read_csv('./processed/test.csv')

In [199]:
trainData.shape

(159343, 10)

In [200]:
testData.shape

(63875, 10)

In [201]:
trainData.head()

,index,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0000997932d777bf,explanation why edits made username hardcore m...,0,0,0,0,0,0
1,1,1,000103f0d9cfb60f,daww he match background colour im seemingly s...,0,0,0,0,0,0
2,2,2,000113f07ec002fd,hey man im really trying edit war its guy cons...,0,0,0,0,0,0
3,3,3,0001b41b1c6bb37e,more i cant make real suggestion improvement i...,0,0,0,0,0,0
4,4,4,0001d958c54c6e35,you sir hero any chance remember page thats,0,0,0,0,0,0


In [202]:
testData.head()

,index,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,5,0001ea8717f6de06,thank understanding i think highly would rever...,0.0,0.0,0.0,0.0,0.0,0.0
1,1,7,000247e83dcc1211,dear god site horrible,0.0,0.0,0.0,0.0,0.0,0.0
2,2,11,0002f87b16116a7f,somebody invariably try add religion reallyou ...,0.0,0.0,0.0,0.0,0.0,0.0
3,3,13,0003e1cccfd5a40a,it say right is type the type institutioneeded...,0.0,0.0,0.0,0.0,0.0,0.0
4,4,14,00059ace3e3e9a53,before adding new product list make surelevant...,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
###### Generate the word2vec embeddings

from gensim.models import Word2Vec

if not os.path.exists('./processed/w2vEmbeddings.model'):
    word2vec = Word2Vec(sentences=trainData['comment_text'].apply(lambda x : nltk.word_tokenize(x)), vector_size=100, window=5, min_count=1, workers=10)
    word2vec.wv.save('./processed/w2vEmbeddings.model')

In [185]:
countVectorizerParams = {
    # 'vectorizer__max_df': (0.5, 1.0),
    # 'vectorizer__ngram_range': ((1, 1), (1, 2)),
}

tfidfVectorizerParams = {
    # 'vectorizer__strip_accents' : ('ascii', 'unicode'),
}

logisticRegressionParams = {
    # 'classifier__solver':['newton-cg', 'saga'],
}

mnbClassifierParams = {
    # 'classifier__alpha' : [.2,.5,1],
}

rfClassifierParams = {
    # 'classifier__max_depth' : [30,40],
}

dtClassifierParams = {
    # 'classifier__splitter' : ["best", "random"]
}


In [196]:
tuples = {
    "transformers": [
        {
            "title": "TfIDF Vectorizer",
            "tuples": [
                ('vectorizer', TfidfVectorizer(analyzer='word'))
            ],
            "params": {
                **tfidfVectorizerParams,
            }
        },
        {
            "title": "Count Vectorizer",
            "tuples": [
                ('vectorizer', CountVectorizer(analyzer='word')),
            ],
            "params": {
                **countVectorizerParams,
            }
        },
        {
            "title": "Paragraph Vectorizer",
            "tuples": [
                ('vectorizer', ParagraphVectorizer()),
            ],
            "params": {}
        }
    ],
    "classifiers": [
        # {
        #     "title" : "Multinomial Naive Bayes",
        #     "tuples" : [
        #         ('scaler',MinMaxScaler()),
        #         ('classifier',MultinomialNB())
        #     ],
        #     "params" : {
        #         **mnbClassifierParams
        #     }
        # },
        {
            "title": "Logistic Regression",
            "tuples": [
                ('classifier', MultiOutputClassifier(LogisticRegression()))
            ],
            "params": {
                **logisticRegressionParams
            }
        },
        {
            "title": "Random Forest Classifier",
            "tuples": [
                ('classifier', MultiOutputClassifier(RandomForestClassifier()))
            ],
            "params": {
                **rfClassifierParams
            }
        },
        {
            "title": "Decision Tree Classifier",
            "tuples": [
                ('classifier', MultiOutputClassifier(DecisionTreeClassifier()))
            ],
            "params": {
                **dtClassifierParams
            }
        }
    ]
}


In [205]:

labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
best_models =[]

for transformer in tuples["transformers"]:
    transformerName = transformer["title"]
    transformerTuple = transformer["tuples"]
    transformerParams = transformer["params"]
    for classifier in tuples["classifiers"]:
        classifierName = classifier["title"]
        classifierTuples = classifier["tuples"]
        classifierParams = classifier["params"]

        print(f' Running {transformerName} + {classifierName}')

        finalParams = {
            **transformerParams,
            **classifierParams
        }

        pipeline = Pipeline(
            transformerTuple
            + classifierTuples
        )

        searchCV = GridSearchCV(
            pipeline, 
            finalParams, 
            cv=3,
            n_jobs=-1, 
            verbose=1,
            error_score='raise'
        )

        name = " ".join([transformerName,classifierName])
        codeName = re.sub(r'[ ]+',"_",name)
        codeName = codeName.lower()
        

        if not os.path.exists(f'./dumps'):
            os.makedirs('./dumps')

        if os.path.exists(f'./dumps/{codeName}.bin'):
            print(f"{name} is already trained")
            continue


        searchCV.fit(dataset['comment_text'],dataset[labels])

        bestEstimator = searchCV.best_estimator_
        bestScore = searchCV.best_score_
        bestParams = searchCV.best_params_

        best_models.append((f'{transformerName} & {classifierName}',bestEstimator,bestScore,bestParams))

        print(f"{name} score : {bestScore}")

        joblib.dump(bestEstimator,f'./dumps/{codeName}.bin')

 Running TfIDF Vectorizer + Logistic Regression
 Running TfIDF Vectorizer + Random Forest Classifier
 Running TfIDF Vectorizer + Decision Tree Classifier
 Running Count Vectorizer + Logistic Regression
 Running Count Vectorizer + Random Forest Classifier
 Running Count Vectorizer + Decision Tree Classifier
 Running Paragraph Vectorizer + Logistic Regression
 Running Paragraph Vectorizer + Random Forest Classifier
 Running Paragraph Vectorizer + Decision Tree Classifier


In [100]:
def explain(name,score,params):
    print("Search for CV "+name)
    print("Best score: %0.3f" % score)
    best_parameters = params
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

for (name,estimator,score,param) in best_models:
    explain(name,score,param)


Search for CV ParagraphVectorizer&LogisticRegression
Best score: 0.873


In [101]:
bestest_estimator = max(best_models, key=lambda tup: tup[2])
bestest_estimator

('ParagraphVectorizer&LogisticRegression',
 Pipeline(steps=[('cleaner', TextCleaner()),
                 ('vectorizer', ParagraphVectorizer()),
                 ('classifier',
                  MultiOutputClassifier(estimator=LogisticRegression()))]),
 0.872983762205319,
 {})

In [102]:
joblib.dump(bestest_estimator[1], bestest_estimator[0].replace('&','_')+'.bin')

['ParagraphVectorizer_LogisticRegression.bin']

In [103]:
test_data = pd.read_csv('./data/test.csv')
test_labels = pd.read_csv('./data/test_labels.csv')


for index, row in test_data.iterrows():
    id = row.id
    index = row.name
    targetRow = test_labels.iloc[index]
    if row.id != targetRow.id:
        print("Not matching")


    test_data.at[index,'toxic'] = int(targetRow.toxic)
    test_data.at[index,'severe_toxic'] = int(targetRow.severe_toxic)
    test_data.at[index,'obscene'] = int(targetRow.obscene)
    test_data.at[index,'threat'] = int(targetRow.threat)
    test_data.at[index,'insult'] = int(targetRow.insult)
    test_data.at[index,'identity_hate'] = int(targetRow.identity_hate)


test_data = test_data.drop(test_data[test_data.toxic == -1].index)
test_data['y'] = test_data.apply(lambda x : transformY(x),axis=1)

test_data.head(10)
test_data.shape

(63978, 9)

In [104]:
# predictive_model = joblib.load('CountVectorizer_LogisticRegression.bin')

bestest_estimator[1]

Pipeline(steps=[('cleaner', TextCleaner()),
                ('vectorizer', ParagraphVectorizer()),
                ('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

In [112]:
predictive_model = bestest_estimator[1]
slimmedTestData = test_data.head(1000)
predictions = pd.DataFrame(predictive_model.predict(slimmedTestData.comment_text),columns=labels)
multilabel_confusion_matrix(slimmedTestData[labels],predictions)

Cleaninig it up...


array([[[851,  58],
        [ 31,  60]],

       [[987,   7],
        [  5,   1]],

       [[914,  29],
        [ 20,  37]],

       [[996,   1],
        [  3,   0]],

       [[906,  35],
        [ 24,  35]],

       [[969,  13],
        [ 14,   4]]])

array([0, 0, 0, ..., 0, 0, 0])

In [90]:
def break_predicted_y(prediction:list):
  #toxic 	severe_toxic 	obscene 	threat 	insult 	identity_hate
  binary_values = [list(bin(y).split('0b')[1]) for y in prediction]
  binary_values = [(['0']*(6-len(x))) + x for x in binary_values]
  predicted_toxic = []
  predicted_severe_toxic = []
  predicted_obscene = []
  predicted_threat = []
  predicted_insult = []
  predicted_identity_hate = []
  for x in binary_values:
    predicted_toxic.append(int(x[0]))
    predicted_severe_toxic.append(int(x[1]))
    predicted_obscene.append(int(x[2]))
    predicted_threat.append(int(x[3]))
    predicted_insult.append(int(x[4]))
    predicted_identity_hate.append(int(x[5]))

  return pd.DataFrame({
      'predicted_toxic': predicted_toxic,
      'predicted_severe_toxic': predicted_severe_toxic,
      'predicted_obscene': predicted_obscene,
      'predicted_threat': predicted_threat,
      'predicted_insult': predicted_insult,
      'predicted_identity_hate': predicted_identity_hate
  })


In [91]:
predicted_df = break_predicted_y(list(predicted_y))
predicted_df

,predicted_toxic,predicted_severe_toxic,predicted_obscene,predicted_threat,predicted_insult,predicted_identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
153159,0,0,0,0,0,0
153160,0,0,0,0,0,0
153161,0,0,0,0,0,0
153162,0,0,0,0,0,0


In [92]:
final_prediction = pd.concat([test_data,predicted_df], axis=1)
final_prediction

,id,comment_text,predicted_toxic,predicted_severe_toxic,predicted_obscene,predicted_threat,predicted_insult,predicted_identity_hate
0,00001cee341fdb12,yo bitch ja rule succesful youll ever whats ha...,0,0,0,0,0,0
1,0000247867823ef7,rfc title fine imo,0,0,0,0,0,0
2,00013b17ad220c46,source zawe ashton lapland,0,0,0,0,0,0
3,00017563c3f7919a,look back source information updated correct f...,0,0,0,0,0,0
4,00017695ad8997eb,dont anonymously edit article,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,totally agree stuff nothing long crap,0,0,0,0,0,0
153160,fffd7a9a6eb32c16,throw field home plate get faster throwing cut...,0,0,0,0,0,0
153161,fffda9e8d6fafa9e,okinotorishima category see change agree corre...,0,0,0,0,0,0
153162,fffe8f1340a79fc2,one founding nation eu germany law return quit...,0,0,0,0,0,0


In [93]:
final_prediction.dtypes

id                         object
comment_text               object
predicted_toxic             int64
predicted_severe_toxic      int64
predicted_obscene           int64
predicted_threat            int64
predicted_insult            int64
predicted_identity_hate     int64
dtype: object

In [94]:
mask = (final_prediction['predicted_toxic'] +	final_prediction['predicted_severe_toxic'] +	final_prediction['predicted_obscene'] +	final_prediction['predicted_threat'] +	final_prediction['predicted_insult'] +	final_prediction['predicted_identity_hate']) > 0
final_prediction[mask]

,id,comment_text,predicted_toxic,predicted_severe_toxic,predicted_obscene,predicted_threat,predicted_insult,predicted_identity_hate
59,0017d4d47894af05,fuck anti semitic cunt,0,1,0,1,1,1
126,0031db73bf0939c3,fuck ur family,0,1,0,1,0,1
136,0038a47d8f1bb327,suck suck suck suck sucku suck suck suck suck ...,0,1,0,1,1,1
189,00500c89c524d462,horny leader earthgirls easy fuck,0,1,0,1,0,1
193,0052e5508d893965,go fuck cunt,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...
152748,ff4ae0cb03e213ca,omfg wtf omfg wtf omfg wtf omfg wtf omfg wtf o...,0,0,0,0,0,1
152953,ffa53a5948abe71d,cunt cunt cunt cunt cunt cunt cunt cunt cunt c...,0,1,0,1,1,1
152992,ffb6ae2411eaaad6,fuck,0,1,0,1,0,1
153044,ffd11cb78532be9b,fuck fucking dirt jewish arab,0,1,0,1,0,1


In [95]:
mask = final_prediction['predicted_toxic'] > 0
final_prediction[mask]

,id,comment_text,predicted_toxic,predicted_severe_toxic,predicted_obscene,predicted_threat,predicted_insult,predicted_identity_hate
7854,0d32a9ad4a5afb50,nigger nigger nigger nigger nigrgers nigger ni...,1,1,0,1,0,1
8067,0d8d8d475b4b676a,die die die die,1,1,1,1,1,1
14428,18384a5942f2adee,die die die die die die die die die die die di...,1,1,1,1,1,1
17058,1ca0189eadeb1443,shimgray filthy cunt shimgray shitty mod shimg...,1,1,0,1,1,1
17328,1d13cc9323658473,ayn rand nigger cock hitler holocaust ayn rand...,1,1,0,1,0,1
18330,1ed7487f6c48ce7d,nigger nigger nigger nigger nigger nigger nigg...,1,1,0,1,0,1
22586,25ba31a890cd1287,dumbass faggot nigger dumbass faggot nigger du...,1,1,0,1,1,1
22592,25c14d76b06ba121,nigger nigger nigger nigger nigger nigger nigg...,1,1,0,1,0,1
26893,2cbcca660647c366,nigger nigger nigger nigger,1,1,0,1,0,1
30029,31f8b3c3985138df,unblock nigger nigger nigger nigger nigger nig...,1,1,0,1,0,1


In [119]:
inp = 'Hey there There there no there'
output = re.sub(r'(\w+)( \1)+', r'\1', inp, flags=re.IGNORECASE)
print(output)  # H

Hey there no there
